In [ ]:
import os
import promptflow as pf
from promptflow.connections import AzureOpenAIConnection
from promptflow.sdk.entities import BulkFlowRunInput

flows_dir = "."
data_path = f"{flows_dir}/webClassification3.jsonl"
connections = {
    "azure_open_ai_connection": AzureOpenAIConnection(
        api_key=os.environ["AOAI_API_KEY"],
        api_base=os.environ["AOAI_API_BASE"],
        api_type="azure",
        api_version="2023-03-15-preview",
    )
}

In [ ]:
flow = pf.load_flow(f"{flows_dir}/web_classification")
flow1 = pf.load_flow(f"{flows_dir}/web_classification_v1")
flow2 = pf.load_flow(f"{flows_dir}/web_classification_v2")

In [ ]:
# TODO pretty print
print(flow)

In [ ]:
baseline = flow.run_bulk(data_path, connections=connections)
v1 = flow1.run_bulk(data_path, connections=connections)
v2 = flow2.run_bulk(data_path, connections=connections)

In [ ]:
# TODO pretty print
print(baseline)

In [ ]:
# TODO fix this
pf.show_details(baseline)

In [ ]:
# TODO fix this
pf.show_metrics(baseline)

In [ ]:
classification_accuracy_eval = pf.load_flow(
    f"{flows_dir}/classification_accuracy_evaluation"
)

inputs_mapping = {"groundtruth": "data.answer", "prediction": "variants.output.category"}
baseline_accuracy_eval_input = BulkFlowRunInput(data_path, [baseline], inputs_mapping=inputs_mapping)
baseline_accuracy = classification_accuracy_eval.run_bulk(baseline_accuracy_eval_input, connections=connections)

v1_accuracy_eval_input = BulkFlowRunInput(data_path, [v1], inputs_mapping=inputs_mapping)
v1_accuracy = classification_accuracy_eval.run_bulk(v1_accuracy_eval_input, connections=connections)

v2_accuracy_eval_input = BulkFlowRunInput(data_path, [v2], inputs_mapping=inputs_mapping)
v2_accuracy = classification_accuracy_eval.run_bulk(v2_accuracy_eval_input, connections=connections)

In [ ]:
accuracy_compare = {
    "baseline": baseline_accuracy["metrics"]["accuracy"],
    "v1": v1_accuracy["metrics"]["accuracy"],
    "v2": v2_accuracy["metrics"]["accuracy"],
}

print(accuracy_compare)

In [ ]:
pf.show_metrics(baseline, v1, v2)

In [ ]:
pf.show_details(baseline, v1, v2)